<a href="https://colab.research.google.com/github/ryotaro1218/project1_2/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np


# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["Canada",  "France", "Germany", "Italy", "Japan", "United Kingdom", "United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh', 'rtfpna', 'pop', 'avh', "country"]]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker
data['a'] = 1 - data['labsh']  # Capital share

# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['g'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita
data['k'] = data['g'] * data['a']
data['tfp'] = data['g'] - data['k']

# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('country').agg({'g': 'mean', 'a': "mean", 'k': "mean", 'tfp': "mean", 'labsh': "mean"})
# Calculate additional summary statistics
summary['Growth Rate'] = summary['g']
summary['TFP Growth'] = summary['tfp']
summary['Capital Growth'] = summary['k']
summary['TFP Share'] = summary['labsh']
summary['Capital Share'] = summary['a']

summary.loc['Average'] = np.nan
summary.loc['Average', :] = summary.mean()

rounded_summary = summary.round(2)

# Print output
print(rounded_summary)

                   g     a     k   tfp  labsh  Growth Rate  TFP Growth  \
country                                                                  
Canada          0.89  0.35  0.31  0.58   0.65         0.89        0.58   
France          0.83  0.38  0.32  0.51   0.62         0.83        0.51   
Germany         0.66  0.37  0.25  0.41   0.63         0.66        0.41   
Italy          -0.04  0.48 -0.02 -0.02   0.52        -0.04       -0.02   
Japan           0.71  0.43  0.31  0.41   0.57         0.71        0.41   
United Kingdom  1.02  0.41  0.43  0.59   0.59         1.02        0.59   
United States   1.54  0.39  0.60  0.94   0.61         1.54        0.94   
Average         0.80  0.40  0.31  0.49   0.60         0.80        0.49   

                Capital Growth  TFP Share  Capital Share  
country                                                   
Canada                    0.31       0.65           0.35  
France                    0.32       0.62           0.38  
Germany                